In [1]:
from core.constants import *
from core.fetch import Fetcher

get_all_sub_tables = lambda table_key: list(vars(table_key)['_value2member_map_'].keys())

fetcher = Fetcher()



In [2]:
import pandas as pd
df_company_profile = pd.read_csv('../data/company-profile-20220827.csv')

In [7]:
df_company_profile['symbol'].duplicated().any()

False

In [3]:
from core.fetch import DATE_KEY
df_company_profile.columns.tolist().count(DATE_KEY)

0

In [22]:
df_company_profile.index = [0] * len(df_company_profile)

In [6]:
df_company_profile.pivot(index=None, columns=TableKey.Profile.SYMBOL, values=TableKey.Profile.SECTOR).index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            493, 494, 495, 496, 497, 498, 499, 500, 501, 502],
           dtype='int64', length=503)

In [10]:
df_company_profile.columns.tolist().count(TableKey.Profile.SYMBOL)
essential_keys = [TableKey.Profile.SYMBOL, ]
all(x in df_company_profile.columns.tolist() for x in ['a', 'b'])

1